<a href="https://colab.research.google.com/github/lucR8/nlp-llm-email-routing/blob/main/notebook_google_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Project — Email Routing (5 departments)

This notebook is the **main deliverable**. It loads the dataset via `datapreparation.py`, trains a **DistilBERT** classifier, and reports **Accuracy**, **Inference Time**, and **Memory**.


## 0. Environment / GPU check

In [3]:
import os, time, json, random
from pathlib import Path

import numpy as np
import torch
import psutil

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

# Paths
ROOT = Path('.').resolve()
OUT_METRICS = ROOT / 'outputs' / 'metrics'
OUT_FIG = ROOT / 'outputs' / 'figures'
OUT_METRICS.mkdir(parents=True, exist_ok=True)
OUT_FIG.mkdir(parents=True, exist_ok=True)



# --------------------------------------------------
# Device selection (CPU / GPU)
# --------------------------------------------------

# 1) Manual override (EDIT HERE IF NEEDED)
MANUAL_DEVICE = "cuda"    # "cpu", "cuda", or None
# MANUAL_DEVICE = "cuda"
# MANUAL_DEVICE = None


# 2) Environment variable
ENV_DEVICE = os.environ.get("ROUTER_DEVICE", "auto").lower()


#  Resolution logic
if MANUAL_DEVICE is not None:
    DEVICE_STR = MANUAL_DEVICE.lower()
else:
    DEVICE_STR = ENV_DEVICE

if DEVICE_STR == "auto":
    DEVICE_STR = "cuda" if torch.cuda.is_available() else "cpu"

if DEVICE_STR == "cuda" and not torch.cuda.is_available():
    print("⚠️ CUDA requested but not available, falling back to CPU.")
    DEVICE_STR = "cpu"

assert DEVICE_STR in {"cpu", "cuda"}, f"Invalid device: {DEVICE_STR}"

device = torch.device(DEVICE_STR)

print(f"Device: {device}")
print("CUDA available:", torch.cuda.is_available())
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

def ram_mb() -> float:
    return psutil.Process(os.getpid()).memory_info().rss / (1024**2)


⚠️ CUDA requested but not available, falling back to CPU.
Device: cpu
CUDA available: False


## 1. Dataset and problem setup

Goal: **route an incoming customer email** to one of 5 departments (*queues*):
1) Billing and Payments  
2) Customer Service  
3) General Inquiry  
4) Sales and Pre‑Sales  
5) Technical Support

We use the dataset `Tobi-Bueck/customer-support-tickets` and focus on the `subject` + `body` fields as input text.


In [4]:
from datapreparation import load_and_prepare_data

train_ds, val_ds, test_ds, label_list, label2id, id2label = load_and_prepare_data()

print('Labels:', label_list)
print('Train size:', len(train_ds), '| Val size:', len(val_ds), '| Test size:', len(test_ds))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

aa_dataset-tickets-multi-lang-5-2-50-ver(…):   0%|          | 0.00/26.0M [00:00<?, ?B/s]

(…)set-tickets-german_normalized_50_5_2.csv: 0.00B [00:00, ?B/s]

dataset-tickets-multi-lang-4-20k.csv:   0%|          | 0.00/18.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/61765 [00:00<?, ? examples/s]

Filter:   0%|          | 0/61765 [00:00<?, ? examples/s]

Filter:   0%|          | 0/28261 [00:00<?, ? examples/s]

Label distribution (train):
Counter({'Technical Support': 6476, 'Customer Service': 3471, 'Billing and Payments': 2307, 'Sales and Pre-Sales': 655, 'General Inquiry': 340})
Labels: ['Billing and Payments', 'Customer Service', 'General Inquiry', 'Sales and Pre-Sales', 'Technical Support']
Train size: 13249 | Val size: 1656 | Test size: 1657


### Sanity checks

We inspect:
- label distribution (to spot class imbalance),
- a few examples (to confirm the fields are correctly loaded),
- dataset sizes after the train/val/test split.


In [5]:
from collections import Counter

print('Train label distribution:')
print(Counter(train_ds['queue']))

# Show a few samples
for i in range(3):
    ex = train_ds[i]
    print('\n--- Example', i, '---')
    print('queue:', ex.get('queue'))
    print('subject:', (ex.get('subject') or '')[:120])
    print('body:', (ex.get('body') or '')[:200])


Train label distribution:
Counter({'Technical Support': 6476, 'Customer Service': 3471, 'Billing and Payments': 2307, 'Sales and Pre-Sales': 655, 'General Inquiry': 340})

--- Example 0 ---
queue: Customer Service
subject: Guidance on Investment Data Analytics
body: Is it possible to receive guidance on optimizing investments through the use of data analytics and available tools and services? I am interested in learning how to make data-driven decisions.

--- Example 1 ---
queue: Sales and Pre-Sales
subject: 
body: Dear customer support, the data analytics tool is failing to process investment data efficiently. The problem might be due to software compatibility issues. After updating the associated software devi

--- Example 2 ---
queue: Customer Service
subject: Concern Regarding CRM System Malfunction
body: Dear Support Team, our marketing agency is facing issues with the Salesforce CRM system, which is disrupting our client data management process. It seems that recent software upda

## 2. Text formatting and labels

We build a single text input:
`Subject: ... \n Body: ...`

We also create `label2id` / `id2label` so that all agents share the same label mapping.


In [6]:
def format_text(ex):
    subject = ex.get('subject', '') or ''
    body = ex.get('body', '') or ''
    return f"Subject: {subject}\nBody: {body}"

def add_text_and_label(ds):
    def _map(ex):
        ex['text'] = format_text(ex)
        ex['label'] = label2id[ex['queue']]
        return ex
    return ds.map(_map)

train_ds2 = add_text_and_label(train_ds)
val_ds2   = add_text_and_label(val_ds)
test_ds2  = add_text_and_label(test_ds)

print('Columns:', train_ds2.column_names)
print('Example fields:', {k: train_ds2[0][k] for k in ['queue','label','text']})


Map:   0%|          | 0/13249 [00:00<?, ? examples/s]

Map:   0%|          | 0/1656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1657 [00:00<?, ? examples/s]

Columns: ['subject', 'body', 'answer', 'type', 'queue', 'priority', 'language', 'version', 'tag_1', 'tag_2', 'tag_3', 'tag_4', 'tag_5', 'tag_6', 'tag_7', 'tag_8', 'text', 'label']
Example fields: {'queue': 'Customer Service', 'label': 1, 'text': 'Subject: Guidance on Investment Data Analytics\nBody: Is it possible to receive guidance on optimizing investments through the use of data analytics and available tools and services? I am interested in learning how to make data-driven decisions.'}


## 3. Agent 1 — GPT‑2 prompting (no training baseline)

We use `distilgpt2` to *generate* a department name given an email, and then parse the generation into one of the 5 labels.

Key constraints for reproducibility:
- deterministic decoding (temperature=0, no sampling),
- short generation (`max_new_tokens` small),
- robust parsing (normalization + synonym matching).


In [7]:
from pathlib import Path

# Project root (safe fallback)
ROOT = globals().get("ROOT", Path(".").resolve())

agents_dir = ROOT / "src" / "agents"
assert agents_dir.exists(), f"Missing agents directory: {agents_dir}"

required = ["gpt2_prompting.py", "gpt2_lora.py", "distilbert_classifier.py"]
for name in required:
    p = agents_dir / name
    assert p.exists(), f"Missing agent file: {p}"

print("[OK] Found agent files in", agents_dir)


[OK] Found agent files in /content/src/agents


In [8]:
from src.agents.gpt2_prompting import GPT2PromptingRouter, PromptingConfig
from src.eval_utils import ram_mb, timed_predict, eval_classification

test_items = [{"subject": ex.get("subject",""), "body": ex.get("body","")} for ex in test_ds]
y_true = [label2id[ex["queue"]] for ex in test_ds]

router = GPT2PromptingRouter(
    label_list=label_list,
    cfg=PromptingConfig(
        model_name="distilgpt2",
        device=DEVICE_STR,
        max_new_tokens=8,
        temperature=0.0,
        do_sample=False,
    )
)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
# Sanity check: raw GPT-2 generations
assert "test_items" in globals(), "Run the data loading cell first (test_items not defined)."

sample = test_items[:10]
raw_outputs = router.generate_raw(sample)

assert raw_outputs is not None, "generate_raw returned None"

for i, out in enumerate(raw_outputs):
    print(f"{i} -> {repr(out)}")


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


0 -> 'The ticket is not a ticket.\n'
1 -> 'The following is a list of the most'
2 -> 'The dashboard is unable to load essential metrics'
3 -> 'The following is a list of the most'
4 -> 'The following is a list of the affected'
5 -> 'The customer service is not available for the'
6 -> 'The following information is provided for you:'
7 -> '\\n<br>\\nWe'
8 -> 'The following is a list of the issues'
9 -> 'Email:\nEmail:\nEmail:'


In [10]:
# Check parsing robustness
from src.agents.gpt2_prompting import parse_department

assert "raw_outputs" in globals(), "raw_outputs not defined. Run the previous sanity-check cell first."

for r in raw_outputs:
    print(r, "=>", parse_department(r, label_list))


The ticket is not a ticket.
 => None
The following is a list of the most => None
The dashboard is unable to load essential metrics => None
The following is a list of the most => None
The following is a list of the affected => None
The customer service is not available for the => None
The following information is provided for you: => None
\n<br>\nWe => None
The following is a list of the issues => None
Email:
Email:
Email: => None


In [ ]:
# Quick eval on 200 samples
timed = timed_predict(router.predict_batch, test_items[:200])
metrics_200 = eval_classification(y_true[:200], timed["preds"], label_list)

print("Accuracy @200:", metrics_200["accuracy"])
print(metrics_200["report"])

In [ ]:
ram_before = ram_mb()
timed = timed_predict(router.predict_batch, test_items[:200])  # start with 200 for speed
ram_after = ram_mb()

y_pred = timed["preds"]
metrics = eval_classification(y_true[:200], y_pred, label_list)

result_prompting = {
    "model": "distilgpt2",
    "agent": "GPT-2 prompting (CPU)",
    "accuracy": metrics["accuracy"],
    "inference_time_sec_total": timed["total_sec"],
    "inference_time_sec_per_item": timed["per_item_sec"],
    "ram_before_mb": ram_before,
    "ram_after_mb": ram_after,
    "ram_delta_mb": ram_after - ram_before,
    "n_test": timed["n_items"],
}

result_prompting, metrics["report"]

**Interpretation:** GPT-2 prompting is a *zero-/few-shot generative baseline*.  
It often has **higher latency and RAM usage** than a small classifier, and accuracy can be low because GPT-2 is not instruction-tuned.

## 4. Agent 2 — GPT‑2 + LoRA (parameter‑efficient fine‑tuning)

In this agent, we turn **GPT‑2 (distilgpt2)** into a **sequence classifier** and fine‑tune only a small number of LoRA parameters.
This is much lighter than full fine‑tuning, and it remains feasible on CPU (GPU helps a lot for speed).


### What we train vs what we keep frozen

- **Frozen:** the base GPT‑2 weights (the 80M parameters stay unchanged)
- **Trainable:** LoRA adapters injected into the attention/projection layers (`c_attn`, `c_proj`)
- **Output:** a multi‑class classifier over the 5 queues

We evaluate the LoRA agent with the same pipeline as Agent 1 (same dataset split and same metrics) to keep results comparable.


In [ ]:
from src.agents.gpt2_lora import GPT2LoRARouter, LoRAConfig, TrainConfig

router_lora = GPT2LoRARouter(
    label_list=label_list,
    cfg=LoRAConfig(
        model_name="distilgpt2",
        device=DEVICE_STR,
        max_length=256,
        r=8,
        alpha=16,
        dropout=0.05,
    )
)

train_info = router_lora.train(
    train_ds=train_ds,
    val_ds=val_ds,
    test_ds=test_ds,
    train_cfg=TrainConfig(
        output_dir=str(ROOT / "outputs" / "checkpoints" / "gpt2_lora"),
        num_train_epochs=2,   # puis 3-4 sur GPU
        per_device_train_batch_size=4,
        per_device_eval_batch_size=8,
    ),
    label2id=label2id,
)

train_info


In [ ]:
from src.eval_utils import ram_mb, timed_predict, eval_classification
from src.agents.gpt2_lora import GPT2LoRARouter, LoRAConfig

# Reuse the same test_items / y_true built earlier to keep evaluation comparable across agents
assert "test_items" in globals(), "test_items not defined. Run the data preparation / prompting setup cells first."
assert "y_true" in globals(), "y_true not defined. Run the data preparation / prompting setup cells first."

router_lora = GPT2LoRARouter(
    label_list=label_list,
    cfg=LoRAConfig(
        model_name="distilgpt2",
        device=DEVICE_STR,
    ),
)

# Option: load the best adapter if available
best_adapter = ROOT / "outputs" / "checkpoints" / "gpt2_lora" / "best_adapter"
if best_adapter.exists():
    router_lora.load(best_adapter)

ram_before = ram_mb()
timed = timed_predict(router_lora.predict_batch, test_items[:500])
ram_after = ram_mb()

y_pred = timed["preds"]
metrics = eval_classification(y_true[:500], y_pred, label_list)

result_lora = {
    "model": "distilgpt2",
    "agent": "GPT-2 LoRA",
    "accuracy": metrics["accuracy"],
    "inference_time_sec_total": timed["total_sec"],
    "inference_time_sec_per_item": timed["per_item_sec"],
    "ram_before_mb": ram_before,
    "ram_after_mb": ram_after,
    "ram_delta_mb": ram_after - ram_before,
    "n_test": timed["n_items"],
}

result_lora, metrics["report"]


(Setup already performed above — this cell was removed to avoid duplicated variables and kernel confusion.)


## 5. Agent 3 — DistilBERT classifier (supervised fine‑tuning)

This is a discriminative encoder model (DistilBERT) fine‑tuned for 5‑class classification.
It is usually the strongest “classic” baseline for text classification with limited compute.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = 'distilbert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
)

### Tokenization
Important: we set the target column name to `labels` because `Trainer` expects that name.


In [ ]:
def tokenize_batch(batch):
    tok = tokenizer(
        batch['text'],
        truncation=True,
        padding='max_length',
        max_length=256,
    )
    tok['labels'] = batch['label']
    return tok

train_tok = train_ds2.map(tokenize_batch, batched=True, remove_columns=train_ds2.column_names)
val_tok   = val_ds2.map(tokenize_batch, batched=True, remove_columns=val_ds2.column_names)
test_tok  = test_ds2.map(tokenize_batch, batched=True, remove_columns=test_ds2.column_names)

# Torch format helps performance
train_tok.set_format(type='torch')
val_tok.set_format(type='torch')
test_tok.set_format(type='torch')

print('train_tok columns:', train_tok.column_names)


### Training (Transformers v5 compatible)

### Handling class imbalance (important for 5 queues)

The train split is imbalanced (e.g., **General Inquiry** and **Sales & Pre-Sales** are much rarer).  
To make the classifier pay more attention to minority classes, we use **class-weighted cross-entropy**.

We compute weights from the training label frequencies and plug them into a custom `Trainer` loss.

In [ ]:
import inspect
from transformers import TrainingArguments

def make_training_args(**kwargs):
    """Create TrainingArguments with backward/forward compatibility."""
    sig = inspect.signature(TrainingArguments.__init__)
    params = set(sig.parameters.keys())

    # evaluation_strategy (old) vs eval_strategy (new)
    if "eval_strategy" in params and "evaluation_strategy" in kwargs:
        kwargs["eval_strategy"] = kwargs.pop("evaluation_strategy")
    if "evaluation_strategy" in params and "eval_strategy" in kwargs:
        kwargs["evaluation_strategy"] = kwargs.pop("eval_strategy")

    # Drop unsupported keys (safety)
    clean = {k: v for k, v in kwargs.items() if k in params}
    return TrainingArguments(**clean)

print("TrainingArguments key present:", {
    "evaluation_strategy": "evaluation_strategy" in inspect.signature(TrainingArguments.__init__).parameters,
    "eval_strategy": "eval_strategy" in inspect.signature(TrainingArguments.__init__).parameters,
})

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import torch.nn as nn

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# --- Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, preds)}

# --- Class weights (inverse frequency)
train_labels = np.array(train_ds2["label"])
counts = np.bincount(train_labels, minlength=len(label_list)).astype(np.float32)
freq = counts / counts.sum()
# simple inverse-freq weights, normalized to mean=1
w = (1.0 / np.maximum(freq, 1e-8))
w = w / w.mean()
class_weights = torch.tensor(w, dtype=torch.float32)

print("Train counts per class:", dict(zip(label_list, counts.astype(int))))
print("Class weights:", dict(zip(label_list, w.round(3))))

class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights: torch.Tensor | None = None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Move weights to correct device
        cw = self.class_weights.to(logits.device) if self.class_weights is not None else None
        loss_fct = nn.CrossEntropyLoss(weight=cw)
        loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

args = make_training_args(
    output_dir=str(ROOT / "outputs" / "checkpoints" / "distilbert"),
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,

    num_train_epochs=4,

    # Good defaults for RTX 3050 Laptop (4GB)
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,   # effective batch ~16

    learning_rate=2e-5,
    weight_decay=0.01,

    fp16=torch.cuda.is_available(),
    dataloader_pin_memory=True,
    dataloader_num_workers=2,

    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

trainer = WeightedTrainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    class_weights=class_weights,
)

In [ ]:
trainer.train()


### Evaluation on the test set

After training, we evaluate on the held‑out test split and compute:
- accuracy
- classification report (per‑class precision/recall/F1)
- confusion matrix
- inference time and RAM deltas (to compare agents fairly)


In [ ]:
def evaluate_on_test(trainer: Trainer, test_tok, label_list):
    ram_before = ram_mb()

    t0 = time.perf_counter()
    pred = trainer.predict(test_tok)
    t1 = time.perf_counter()

    logits = pred.predictions
    y_true = pred.label_ids
    y_pred = np.argmax(logits, axis=-1)

    acc = accuracy_score(y_true, y_pred)
    elapsed = t1 - t0

    ram_after = ram_mb()

    report = classification_report(y_true, y_pred, target_names=label_list, zero_division=0)
    cm = confusion_matrix(y_true, y_pred)

    result = {
        'model': MODEL_NAME,
        'agent': 'DistilBERT classifier',
        'accuracy': float(acc),
        'inference_time_sec_total': float(elapsed),
        'inference_time_sec_per_item': float(elapsed / max(1, len(y_true))),
        'ram_before_mb': float(ram_before),
        'ram_after_mb': float(ram_after),
        'ram_delta_mb': float(ram_after - ram_before),
        'n_test': int(len(y_true)),
    }
    return result, report, cm

# Run after training:
result, report, cm = evaluate_on_test(trainer, test_tok, label_list)
print('Accuracy:', result['accuracy'])
print('Inference total (s):', result['inference_time_sec_total'])
print('RAM delta (MB):', result['ram_delta_mb'])
print('\nClassification report:\n', report)
print('\nConfusion matrix:\n', cm)


### Persisting metrics and artifacts

We save:
- a JSON summary in `outputs/metrics/`
- optional figures (confusion matrix) in `outputs/figures/`
- checkpoints in `outputs/checkpoints/`


In [ ]:
# After evaluation:
(OUT_METRICS / 'distilbert_results.json').write_text(json.dumps(result, indent=2), encoding='utf-8')
print('Saved:', OUT_METRICS / 'distilbert_results.json')


## 6. Final comparison (Agent 1 vs Agent 2 vs Agent 3)

We aggregate the metrics in a single table so the professor can quickly compare:
- accuracy
- per‑class precision/recall/F1
- inference speed
- RAM usage


In [ ]:
from __future__ import annotations

import json
import pandas as pd
import matplotlib.pyplot as plt

from src.config import default_paths

paths = default_paths()
paths.metrics.mkdir(parents=True, exist_ok=True)
paths.figures.mkdir(parents=True, exist_ok=True)

summary_path = paths.metrics / "summary_results.csv"
distilbert_json = paths.metrics / "distilbert_results.json"

cols = [
    "model","agent","accuracy","inference_time_sec_total","inference_time_sec_per_item",
    "ram_before_mb","ram_after_mb","ram_delta_mb","n_test"
]

# Load summary (may contain only GPT-2 baselines)
df = pd.read_csv(summary_path) if summary_path.exists() else pd.DataFrame(columns=cols)

# Ensure all required columns exist
for c in cols:
    if c not in df.columns:
        df[c] = pd.NA
df = df[cols]

# Force numeric types where relevant (avoid strings / NA issues)
num_cols = ["accuracy","inference_time_sec_total","inference_time_sec_per_item","ram_before_mb","ram_after_mb","ram_delta_mb","n_test"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# ---- DistilBERT: add OR repair row using distilbert_results.json ----
dist_name = "DistilBERT classifier"

if distilbert_json.exists():
    d = json.loads(distilbert_json.read_text(encoding="utf-8"))

    # Extract from JSON (authoritative)
    dist_row = {
        "model": d.get("model", "distilbert-base-uncased"),
        "agent": d.get("agent", dist_name),
        "accuracy": float(d.get("accuracy", float("nan"))),
        "inference_time_sec_total": float(d.get("inference_time_sec_total", float("nan"))),
        "inference_time_sec_per_item": float(d.get("inference_time_sec_per_item", float("nan"))),
        "ram_before_mb": float(d.get("ram_before_mb", float("nan"))),
        "ram_after_mb": float(d.get("ram_after_mb", float("nan"))),
        "ram_delta_mb": float(d.get("ram_delta_mb", float("nan"))),
        "n_test": int(d.get("n_test", 1657)),
    }

    # If row exists but is incomplete (NaN), repair it
    mask = df["agent"].astype(str) == dist_name
    if mask.any():
        idx = df.index[mask][0]
        for k, v in dist_row.items():
            if pd.isna(df.at[idx, k]) or (k in ["inference_time_sec_total","inference_time_sec_per_item"] and (df.at[idx, k] != df.at[idx, k])):  # NaN check
                df.at[idx, k] = v
    else:
        df = pd.concat([df, pd.DataFrame([dist_row])], ignore_index=True)

# If per-item is still NaN but total exists, compute it
mask = df["agent"].astype(str) == dist_name
if mask.any():
    idx = df.index[mask][0]
    if pd.isna(df.at[idx, "inference_time_sec_per_item"]) and not pd.isna(df.at[idx, "inference_time_sec_total"]):
        n = int(df.at[idx, "n_test"]) if not pd.isna(df.at[idx, "n_test"]) else 1657
        df.at[idx, "inference_time_sec_per_item"] = float(df.at[idx, "inference_time_sec_total"]) / max(1, n)

# Save back
df.to_csv(summary_path, index=False)
display(df)

# Replot comparisons with ALL agents present
plt.figure()
plt.bar(df["agent"], df["accuracy"])
plt.title("Accuracy — comparison of agents")
plt.ylabel("Accuracy")
plt.xticks(rotation=25, ha="right")
plt.tight_layout()
plt.savefig(paths.figures / "comparison_accuracy.png", dpi=150)
plt.show()

plt.figure()
plt.bar(df["agent"], df["inference_time_sec_per_item"])
plt.title("Inference time per item — comparison of agents")
plt.ylabel("Seconds / item")
plt.xticks(rotation=25, ha="right")
plt.tight_layout()
plt.savefig(paths.figures / "comparison_inference_time_per_item.png", dpi=150)
plt.show()

print(f"Saved: {summary_path}")
print(f"Saved figures in: {paths.figures}")


## 7. Discussion and next steps

**What we learned so far**
- DistilBERT (Agent 3) provides a strong supervised baseline for routing, but minority classes (e.g., *General Inquiry*, *Sales and Pre‑Sales*) are harder.
- GPT‑2 prompting (Agent 1) is extremely cheap to set up, but accuracy is typically low without careful prompt design and parsing.
- GPT‑2 + LoRA (Agent 2) is the “middle ground”: it can improve over prompting while staying lighter than full fine‑tuning.